In [1]:
# Get raw data
with open('input/10.txt', 'r') as f:
    rawinput = f.read().strip()

In [2]:
import numpy as np

In [3]:
# Part 1
def get_buttons(x,y):
    buttons = np.zeros((len(x), y), dtype=int)
    buttons[*zip(*[[i,k] 
                   for i,j in enumerate(x) 
                   for k in eval(f'[{j[1:-1]}]')])] = 1
    return buttons

manual = [[(z:=np.array([[int(k=='#') for k in j[0][1:-1]]])), 
           get_buttons(j[1:-1], z.shape[1]), 
           np.array([int(k) for k in j[-1][1:-1].split(',')])]
          for i in rawinput.split('\n') 
          for j in [i.split()]]

hlist = lambda x: x.astype('<U1').view(f'<U{x.shape[1]}').reshape(-1).tolist()

tot_n_press = 0
for req, buttons, _ in manual:
    poss = np.zeros_like(req)
    hist = hlist(poss)
    n_press = 0
    while True:
        n_press += 1
        poss = np.logical_xor(np.repeat(poss, buttons.shape[0], axis=0),
                              np.tile(buttons, (poss.shape[0], 1))).astype(int)
        if np.any(np.all(poss == req, axis=1)):
            break
        poss = poss[((h:=np.array(hlist(poss)))[s:=np.lexsort((np.arange(h.shape[0]), h))] 
                     != np.append([-1], h[s][:-1]))
                    [np.argsort(s)]]
        poss = poss[~np.isin(hlist(poss), hist)]
        hist += hlist(poss)
    tot_n_press += n_press

tot_n_press

491

In [4]:
# Part 2
def divide_cols(work):
    pf = [2]
    while pf[-1] <= np.sqrt(np.max(np.abs(work))):
        while ~np.all(work == (work:=np.where(np.all(np.mod(work, pf[-1]) == 0, 
                                                     axis=0, 
                                                     keepdims=True), 
                                              work//pf[-1], work))):
            pass
        pf += [pf[-1]+1]
        while any([pf[-1] % i == 0 for i in pf[:-1]]):  pf[-1] += 1
    return work

def get_search_values(x):
    sv = np.empty((1,0), dtype=int)
    for i in x:
        sv = np.column_stack((np.repeat(sv, i+1, axis=0), 
                              np.tile(np.arange(i+1), sv.shape[0])))
    return sv

def get_min_presses(r_buttons, r_req):
    # Create matrix combining button mapping and requirements, get max presses for each button
    work = np.append(r_buttons, [r_req], axis=0)
    bpmax = np.nanmin(np.where(r_buttons==0, np.nan, r_buttons * [r_req]), axis=1).astype(int)

    # Gaussian Elimination to reduce search space
    r, c = 0, 0
    while np.all(np.subtract(r_buttons.shape, [r,c]) > 0):
        if (y:=np.abs(work[r, c:]))[(z:=np.argmax(y))] == 0:
            # No pivot - move to next row
            r += 1
        else:
            # Swap columns
            work[:, [c, c+z]] = work[:, [c+z, c]]
            # Multiply to avoid non-integers
            work *= np.where(np.mod(work[r:r+1], work[r:r+1, c]) != 0, work[r:r+1, c], 1)
            # Elimination step
            work -= np.where(np.expand_dims((cix:=np.arange(nc:=work.shape[1])) == c, 0), 
                             0, 
                             work[r:r+1] // work[r:r+1, c] * work[:, c:c+1])
            # Divide each column by max common divisor
            work = divide_cols(work)
            r,c = r+1,c+1
            
    # If needed, sort "free" rows (with >1 non-zero value) to bottom, reorder bpmax to match    
    work = work[np.ix_((six:=np.lexsort(((rix:=np.arange(nr:=work.shape[0])), 
                                         np.sum(work != 0, axis=1) > 1))),
                       (cix:=cix[np.any(work != 0, axis=0)]))]
    bpmax = bpmax[six[:-1]]
    
    # Flip signs in columns with negative pivot values
    work *= np.sign(work[np.min(np.where(work==0, nr-1, np.expand_dims(rix, 1)), axis=0), cix])
    
    if nr - (ptr:=np.argmax(np.sum(work != 0, axis=1) > 1)) - 1:
        # If there are free rows do brute force search through value combos to get min # presses
        return np.min(np.sum(np.append((z:=(work[-1:] 
                                            - np.sum(np.expand_dims((sv:=get_search_values(bpmax[ptr:])), 2) 
                                                     * np.expand_dims(work[ptr:-1], 0), axis=1)) 
                                        / np.max(work[:ptr], axis=0)).astype(int), 
                                       sv, axis=1)[np.all((z >= 0) & (np.mod(z, 1) == 0), axis=1)], axis=1)).item()
    else:
        # Otherwise return total of bottom row
        return np.sum(work[-1]).item()
    
sum([get_min_presses(buttons, req) for _, buttons, req in manual])

20617